In [162]:
from parsel import Selector
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import ElementClickInterceptedException, NoSuchElementException, ElementNotInteractableException
from selenium.webdriver.support import expected_conditions as ec
from selenium.webdriver.common.by import By
import time
import re
import requests
import pandas as pd
import numpy as np

In [166]:
urls_amazon = ['https://amzn.to/2WtM2dZ',
'https://amzn.to/399Z0mi',
'https://amzn.to/2OwpzIP',
'https://amzn.to/3jd5SUi',
'https://amzn.to/2B9rakL',
'https://amzn.to/32upflY',
'https://amzn.to/3eysbAo',
'https://amzn.to/2CGorQe',
'https://amzn.to/2WsnVMy',
'https://amzn.to/3haivxL',
'https://amzn.to/2WwF9IC',]

In [167]:
df = pd.DataFrame(columns = ["urlz_amazon","name", "instituicao", "site_plataforma", "autor", "descricao", "avaliacoes", "paginas", "idioma", "img_name", "modalidade_1", "preco_fisico", "cashback", "site_de_compra", "modalidade_2", "preco_fisico"])
driver = webdriver.Chrome('chromedriver.exe')
for url_amazon in urls_amazon:
    driver.get(url_amazon)
    time.sleep(7)
    sel = Selector(text = driver.page_source)
    try:
        name = driver.find_element_by_id("productTitle").text
    except:
        name = ""
    try:
        autor1 = driver.find_element_by_class_name("author").find_element_by_tag_name("a").text
    except:
        autor1 =""
    try:
        autor2 = driver.find_element_by_class_name("contributorNameID").text
    except:
        autor2= ""
    autor = autor1
    if autor == "":
        autor = autor2
    try:    
        spans = driver.find_element_by_id("detail_bullets_id").find_element_by_class_name("content").text.split("\n")
        regex = re.compile(r'(.*)páginas(.*)')
        paginas = [i for i in spans if regex.match(i)][0].split("\n")[0].split(":")[1].strip()
    except:
        paginas = "--"
    try:
        spans = driver.find_element_by_id("detail_bullets_id").find_element_by_class_name("content").text.split("\n")
        regex = re.compile(r'(.*)Idioma(.*)')
        idioma = [i for i in spans if regex.match(i)][0].split("\n")[0].split(":")[1].strip()
    except:
        idioma  = "--"
    try:
        avaliacoes = sel.xpath('//span[@class = "a-icon-alt"]/text()').get().replace(" de ", "/").replace(" estrelas", "")     
    except:
        avaliacoes = ""

    driver.switch_to.frame(driver.find_element_by_id("bookDesc_iframe")) 
    descricao = driver.find_element_by_id("iframeContent").text
    driver.switch_to.default_content()
    img = driver.find_element_by_class_name("frontImage").get_attribute("src")
    path = 'G:/Meu Drive/BRG - SkillTrack/Design/Imagens das ofertas/'
    img_name = path + "Livro" + url.split("/")[-1] + '.jpg'
    img_data = requests.get(img).content
    with open(img_name, 'wb') as handler:
        handler.write(img_data)
    spans = sel.xpath('//ul[@class = "a-unordered-list a-nostyle a-button-list a-horizontal"]//a[@class = "a-button-text"]//*/text()').getall()
    regex = re.compile(r'Kindle|Capa|\nR\$(.*)')
    filtered = [i for i in spans if regex.match(i)]
    try:    
        preco_digital = filtered[[i for i, word in enumerate(filtered) if re.search("^Kindle$", word)][0]+1].replace("\n", "").replace("R$0,00","-")
    except:
        preco_digital = "-"
    try:
        preco_fisico = filtered[[i for i, word in enumerate(filtered) if re.search("^Capa", word)][0]+1].replace("\n", "")
    except:
        preco_fisico = "-"
    instituicao = "Amazon"
    site_plataforma = "https://www.amazon.com.br/"
    cashback = "10%"
    modalidade_1 = "Fisico"
    modalidade_2 = "Digital"

    print(url_amazon,preco_digital, preco_fisico, autor, paginas)
    
    df = df.append(pd.DataFrame([[url_amazon, name, instituicao, site_plataforma, autor, descricao, avaliacoes, paginas, idioma, img_name, modalidade_1, preco_fisico, cashback, url_amazon, modalidade_2, preco_digital]],columns = df.columns))
        

https://amzn.to/2WtM2dZ - R$90,00 Jeffrey Gitomer 296 páginas
https://amzn.to/399Z0mi R$23,28 R$48,86 Dan Steinman 272 páginas
https://amzn.to/2OwpzIP R$24,99 R$185,83 Mark Organ 246 páginas
https://amzn.to/3jd5SUi R$97,75 R$102,90 Jeanne Bliss 288 páginas
https://amzn.to/2B9rakL R$88,77 R$110,20 Leonardo Inghilleri 189 páginas
https://amzn.to/32upflY R$45,08 R$146,08 Tony Hsieh 272 páginas
https://amzn.to/3eysbAo R$23,66 R$22,68 Tony Hsieh 256 páginas
https://amzn.to/2CGorQe - R$41,29 Fred Reichheld 224 páginas
https://amzn.to/2WsnVMy R$36,55 R$38,88 Damin Hiram 168 páginas
https://amzn.to/3haivxL - - Fabiola Tomaz --
https://amzn.to/2WwF9IC R$119,39 R$125,67 Ashvin Vaidyanathan 288 páginas


In [168]:
df.to_excel("Livros_Amazon-CS.xlsx")